In [1]:
%pylab inline
import control
import pyhull
import string
import picos as pic
import cvxopt as cvx
import numpy as np
import scipy.linalg
import scipy.optimize
import scipy.integrate
import itertools
import sympy
import sympy.physics.mechanics as me
from matplotlib.patches import Ellipse

Populating the interactive namespace from numpy and matplotlib


In [2]:
def R(theta):
    R = np.array([[cos(theta), -sin(theta)], 
                  [sin(theta), cos(theta)]])
    return R

In [3]:
def omega(w_theta, w_l, w_tr):
    omega = np.array([[w_theta],[w_l],[w_tr]])
    return omega

In [76]:
def iekf(vt, x0, P0, dt):
    
    theta = x0[0]
    x1 = x0[1]
    x2 = x0[2]
    # left invariant error in exponential coordinate
    X = array([[cos(theta), -sin(theta), x1],
               [sin(theta), cos(theta), x2],
               [0, 0, 1]])
    
    # Initial Condition
    X_h = X
    P = P0
    Xi = np.zeros([3,1])
    
    t = 1 # time
    step = int(t/dt)
    
    v = vt[0] # velocity measured by an odometer
    w = vt[1] # velocity measured through differential odometry
    
    Out_X = []
    Out_Xi = []
    Out_Xe = []
    
    for i in range(step):
        
        Q = array([[0.1, 0.2, 0.2],
                   [0.5, 0.1, 0.2],
                   [0.3, 0.3, 0.1]])
        w_noise = np.random.normal(0, sqrt(Q), (3,1))
        w_theta = w_noise[0][0] # differential odometry error
        w_l = w_noise[1][0] # longitudinal odometry error
        w_tr = w_noise[2][0] # the transversal shift
        
        A = -array([[0, 0, 0],
                    [0, 0, -w],
                    [-v, w, 0]])
        vt = array([[0, -w, v],
                    [w, 0, 0],
                    [0, 0, 0]])
        wt = array([[0, -w_theta, w_l],
                    [w_theta, 0, w_tr],
                    [0, 0, 0]])
        
        
        # Reference Trajectory
        X = X + X.dot(vt + wt) * dt
        Out_X.append(X)
        
        # Propagation
        dXi = A.dot(Xi) - omega(w_theta, w_l, w_tr)
        dX_h = X_h.dot(vt)
        Xi = Xi + dXi*dt
        X_h = X_h + dX_h*dt

        # Covariance Propagation
        dP = (A.dot(P)+P.dot(A.T)+Q)*dt
        P = P+dP
        
        Vn_cov = array([[0.1, 0.5],
                        [0.2, 0.1]])
        Vn = np.random.normal(0, sqrt(Vn_cov), (2,1))
        # Measurement
        Y = X.dot(array([[0, 0, 1]]).T) + np.append(Vn,[0]).reshape(3,1)

        #iekf Gain
        H = array([[0, 1, 0],
                   [0, 0, 1]])
        R = X[0:2,0:2]
        N = R@Vn_cov@R.T
        S = H@P@H.T + N
        S = inv(S)
        Ln = P.dot(H.T).dot(np.linalg.inv(S)) # Gain

        # update 
        Xi = Xi - Ln.dot(array([[0,1,0],[0,0,1]]).dot(Xi) - R.dot(Vn)) # updated invariant error in exp coordinate
        p = array([[1, 0, 0],[0, 1, 0]])
        zeta = Ln.dot(p).dot(inv(X_h).dot(Y) - array([[0],[0],[1]]))
        alpha = zeta[0][0]
        u1 = zeta[1][0]
        u2 = zeta[2][0]
        zeta_x = array([[sin(alpha)/alpha, -(1-cos(alpha)/alpha)],
                        [(1-cos(alpha)/alpha), sin(alpha)/alpha]]).dot(array([[u1],[u2]]))
        R_alpha = np.array([[cos(alpha), -sin(alpha)], 
                            [sin(alpha), cos(alpha)]])
        exp_zeta = np.vstack((np.hstack((R_alpha, zeta_x)),array([[0,0,1]])))
        X_h = X_h.dot(exp_zeta) # updated state
        Out_Xi.append(Xi)
        Out_Xe.append(X_h)
    
        
    return Out_Xi, Out_Xe

In [77]:
vt = [1,0]
x0 = [0,0,0]
P0 = array([[1, 0, 0],[0, 1, 0],[0, 0, 1]]) # initial covariance?
dt = 0.1
Xi, X = iekf(vt, x0, P0, dt)

In [79]:
X

[array([[ 0.99374116, -0.11170727,  7.53610974],
        [ 0.11170727,  0.99374116,  0.38320324],
        [ 0.        ,  0.        ,  1.        ]]),
 array([[  0.81311969,   0.58209653,   6.492325  ],
        [ -0.58209653,   0.81311969, -19.22938554],
        [  0.        ,   0.        ,   1.        ]]),
 array([[ -0.2454142 ,  -0.96941832, -13.48575326],
        [  0.96941832,  -0.2454142 , -22.36140808],
        [  0.        ,   0.        ,   1.        ]]),
 array([[  0.10334448,  -0.99464562, -30.74918327],
        [  0.99464562,   0.10334448,  -5.86209875],
        [  0.        ,   0.        ,   1.        ]]),
 array([[-0.5193706 , -0.85454911, -9.30718796],
        [ 0.85454911, -0.5193706 , 46.20352323],
        [ 0.        ,  0.        ,  1.        ]]),
 array([[ 0.96318372, -0.26884405, 36.83767928],
        [ 0.26884405,  0.96318372, 48.66570254],
        [ 0.        ,  0.        ,  1.        ]]),
 array([[ -0.28761569,   0.9577459 , 215.72680916],
        [ -0.9577459 ,  -0.

In [80]:
Xi

[array([[0.10318938],
        [0.14749915],
        [0.89906962]]), array([[-0.13078429],
        [-0.43903734],
        [-0.34976796]]), array([[0.13454315],
        [0.2500251 ],
        [0.36447343]]), array([[ 0.61657203],
        [-0.20560959],
        [ 1.38030144]]), array([[-1.57703211],
        [-2.15107489],
        [-3.12780758]]), array([[3.40498427],
        [4.1256278 ],
        [6.29298659]]), array([[-10.47551925],
        [-11.78248747],
        [-19.17142321]]), array([[32.58993662],
        [31.97171034],
        [57.91462254]]), array([[-141.06317699],
        [-120.9258513 ],
        [-252.87283161]]), array([[ 777.91845455],
        [ 607.35995949],
        [1404.68507635]])]

In [169]:
    Vn = array([[5],[1]])

In [170]:
np.append(Vn,[0]).reshape(3,1)

array([[5],
       [1],
       [0]])

In [171]:
cov(Vn)

/Users/LiYuLin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: Degrees of freedom <= 0 for slice
  """Entry point for launching an IPython kernel.


array([[nan, nan],
       [nan, nan]])

In [181]:
cov(array([[w_theta,w_l,w_tr]]).T,array([[w_theta,w_l,w_tr]]).T)

/Users/LiYuLin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: Degrees of freedom <= 0 for slice
  """Entry point for launching an IPython kernel.


array([[nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan]])

In [177]:
w_theta = 1 # differential odometry error
w_l = 2 # longitudinal odometry error
w_tr = 1 # the transversal shift

In [49]:
x =array([[w_theta],[w_l],[w_tr]]).T
Q = var(x)
   

In [50]:
w_theta = 1 # differential odometry error
w_l = 2 # longitudinal odometry error
w_tr = 1 # the transversal shift

In [52]:
Q = diag(Q)

ValueError: Input must be 1- or 2-d.